In [28]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
# Part 1: Calculate projection coefficients and print them

# Define constants
L = 1  # Length of the rope from 0 to 1
c = 1  # Wave speed
nx = 100  # Number of spatial points
dx = L / (nx - 1)  # Spatial step size
dt = 0.5 * dx / c  # Time step size, adjusted to satisfy CFL condition
nt = 1000  # Number of time points
x = np.linspace(0, 1, nx)
n_modes = 20  # Number of normal modes to consider

# Initial condition
u_initial = x**2 * (x - 1)
v = np.zeros_like(x)  # Initial velocity (assuming it starts from rest)

# Function to calculate projection on normal modes
def calculate_projections(u_initial, n_modes, x, L):
    projections = []
    for n in range(1, n_modes + 1):
        k = n * np.pi / L
        phi_n = np.sin(k * x)
        a_n = (2 / L) * np.trapz(u_initial * phi_n, x)
        projections.append((a_n, phi_n, n))
    return projections

# Calculate projections
projections = calculate_projections(u_initial, n_modes, x, L)

# Print projection coefficients
coefficients = [a_n for a_n, _, _ in projections]
print("Projection coefficients:", np.array(coefficients))

# Part 2: Create animation with user-specified coefficients

# Function to generate u_projection_history
def generate_projection_history(coefficients, projections, nt, x, c, L, dt):
    u_projection_history = []
    for t in range(nt):
        u_t = np.zeros_like(x)
        for i, (a_n, phi_n, n) in enumerate(projections):
            if i < len(coefficients):
                a_n = coefficients[i]
            omega_n = n * np.pi * c / L
            u_t += a_n * phi_n * np.cos(omega_n * t * dt)
        u_projection_history.append(u_t)
    return u_projection_history



Projection coefficients: [-1.29006137e-01  4.83772984e-02 -4.77800346e-03  6.04715616e-03
 -1.03204637e-03  1.79174207e-03 -3.76107358e-04  7.55882189e-04
 -1.76958214e-04  3.87001930e-04 -9.69181665e-05  2.23949139e-04
 -5.87120426e-05  1.41018440e-04 -3.82157159e-05  9.44602110e-05
 -2.62486352e-05  6.63309724e-05 -1.87976401e-05  4.83435290e-05]


In [ ]:
# User-specified coefficients (example: using the calculated ones)
# user_coefficients = coefficients  # Change this list to use different coefficients


#####################################
user_coefficients=np.zeros(n_modes)

# user_coefficients=[-0.1290061371876774, 0.04837729837829159, -0.004778003464807031,
#                    0.006047156156929939, -0.0010320463710248948, 0.0017917420732872422, 
#                    -0.0003761073582056154, 0.0007558821892908841, -0.00017695821431017026,
#                    0.00038700192950628573]


#Using 
n_max_used=4
user_coefficients[0:n_max_used]=coefficients[0:n_max_used]

user_coefficients




#####################################



# Generate u_projection_history with user-specified coefficients
u_projection_history = generate_projection_history(user_coefficients, projections, nt, x, c, L, dt)

# Time evolution for the original wave equation solution
u_history = []
u = u_initial.copy()
u_prev = u_initial.copy()

for t in range(nt):
    if t > 0:
        u_new = 2 * u - u_prev + (c * dt / dx) ** 2 * (np.roll(u, -1) - 2 * u + np.roll(u, 1))
        u_new[0] = u_new[-1] = 0  # Fixed ends
        u_prev = u
        u = u_new
    u_history.append(u.copy())

# Create animation
fig = go.Figure(
    frames=[go.Frame(data=[
        go.Scatter(x=x, y=u_history[k], name='Original Solution'),
        go.Scatter(x=x, y=u_projection_history[k], name='Projection on Normal Modes')
    ], name=f'frame{k}') for k in range(nt)],
    layout=go.Layout(
        updatemenus=[dict(type="buttons",
                          buttons=[dict(label="Play",
                                        method="animate",
                                        args=[None, {"frame": {"duration": 6.25, "redraw": True},
                                                     "fromcurrent": True, "mode": "immediate"}]),
                                   dict(label="Pause",
                                        method="animate",
                                        args=[[None], {"frame": {"duration": 0, "redraw": False},
                                                       "mode": "immediate"}])])])
)

fig.update_layout(
    title=f"Wave Propagation with Original Solution and First n Normal Modes",
    xaxis_title="x",
    yaxis_title="Displacement",
    xaxis=dict(range=[0, 1], autorange=False),
    yaxis=dict(range=[-0.2, 0.2], autorange=False),
)

# Add initial data
fig.add_trace(go.Scatter(x=x, y=u_history[0], name='Original Solution'))
fig.add_trace(go.Scatter(x=x, y=u_projection_history[0], name='Projection on Normal Modes'))

# Show the animation
fig.show()


In [ ]:

# Define constants
L = 1  # Length of the rope from 0 to 1
c = 1  # Wave speed
nx = 100  # Number of spatial points
dx = L / (nx - 1)  # Spatial step size
dt = 0.5 * dx / c  # Time step size, adjusted to satisfy CFL condition
nt = 600  # Number of time points
x = np.linspace(0, 1, nx)
max_modes = 20  # Maximum number of normal modes to consider

# User specifies the number of modes
n = 5  # Change this value as needed

# Function to calculate normal modes
def calculate_normal_modes(n, x, L):
    modes = []
    for i in range(1, n + 1):
        k = i * np.pi / L
        phi_n = np.sin(k * x)
        modes.append((phi_n, i))
    return modes

# Function to generate normal mode history
def generate_normal_mode_history(modes, nt, x, c, L, dt):
    normal_mode_history = []
    for t in range(nt):
        u_t_list = []
        for phi_n, n in modes:
            omega_n = n * np.pi * c / L
            u_t = phi_n * np.cos(omega_n * t * dt)
            u_t_list.append(u_t)
        normal_mode_history.append(u_t_list)
    return normal_mode_history

# Generate normal mode history
modes = calculate_normal_modes(n, x, L)
normal_mode_history = generate_normal_mode_history(modes, nt, x, c, L, dt)

# Create subplots
fig = make_subplots(rows=n, cols=1, shared_xaxes=True, vertical_spacing=0.02)

# Add initial data for each subplot
for i in range(n):
    fig.add_trace(go.Scatter(x=x, y=normal_mode_history[0][i], name=f'Normal Mode {i+1}', showlegend=False), row=i+1, col=1)

# Update layout to stack subplots in a single column
fig.update_layout(
    title="Normal Modes Evolution",
    xaxis_title="x",
    yaxis_title="Displacement",
    height=300 * n,  # Adjust the height as needed
    updatemenus=[dict(
        type="buttons",
        buttons=[dict(
            label="Play",
            method="animate",
            args=[None, {"frame": {"duration": 6.25, "redraw": True},
                         "fromcurrent": True, "mode": "immediate"}]),
            dict(label="Pause",
                 method="animate",
                 args=[[None], {"frame": {"duration": 0, "redraw": False},
                                "mode": "immediate"}])])])


# Update axes titles for each subplot
for i in range(n):
    fig.update_xaxes(title_text="x", row=i+1, col=1)
    fig.update_yaxes(title_text=f"Mode {i+1}", row=i+1, col=1, range=[-2, 2])  # Adjust y-axis range as needed

# Create frames for the animation
frames = [go.Frame(
    data=[go.Scatter(x=x, y=normal_mode_history[k][i], name=f'Normal Mode {i+1}', showlegend=False) for i in range(n)],
    name=f'frame{k}'
) for k in range(nt)]

fig.frames = frames

# Display the figure
fig.show()